In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
import ee
import geemap.foliumap as geemap
from google.colab import drive
import pandas as pd
import geopandas as gpd
import pyproj
import pyarrow
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from keras.models import load_model

gpd.options.io_engine = "pyogrio"
os.environ["PYOGRIO_USE_ARROW"] = "1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
ee.Authenticate()
ee.Initialize(project="215656163750")
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import utils
import explore

Mounted at /content/drive


In [ ]:
model_name = "VGG16"

training_data = "data_combo_1"

model = load_model(f'/content/drive/MyDrive/CAFO_models/{model_name}_{training_data}/model_ft.keras')

In [ ]:
data_path = "/content/drive/MyDrive/CAFO_data/forTraining/"

data = pd.read_pickle(f'{data_path}sdn_iter0.pkl')
print(f"There are {len(data)} rows in the Sudan dataset")

There are 241 rows in the Sudan dataset


,ADM1,geometry,Dataset name,Farm type,Sentinel
0,Northern,"POLYGON ((31.69635 16.56403, 31.69648 16.56382...",Sudan,Non-farm,"[[[159.3966434540047, 105.32916491192852, 59.9..."
1,Northern,"POLYGON ((31.7561 16.59074, 31.7561 16.59063, ...",Sudan,Non-farm,"[[[194.47657257659247, 130.85573116454003, 73...."
2,Northern,"POLYGON ((31.61158 17.04083, 31.61164 17.04079...",Sudan,Non-farm,"[[[244.3510896089415, 148.7425196315044, 78.67..."
3,Northern,"POLYGON ((31.41967 18.02514, 31.4197 18.02507,...",Sudan,Non-farm,"[[[195.39575813129483, 135.15770907465375, 82...."
4,Northern,"POLYGON ((31.56502 18.02835, 31.56542 18.02831...",Sudan,Non-farm,"[[[212.23700029774577, 147.9646751113527, 93.8..."


AttributeError: 'DataFrame' object has no attribute 'crs'

In [ ]:
X = []
labels = []

for idx, row in data.iterrows():
  labels.append("Unlabeled")
  X.append(row["Sentinel"])

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
y = to_categorical(encoded_labels, num_classes=len(set(labels)))

class_mapping = dict(zip(encoded_labels, labels))
print("Class mapping:", class_mapping)

results = utils.get_predictions(model, X, y, model_name, data)
results.loc[:, "prob_0"] = results["Model Probabilities"].str[0]
display(results.head())

Class mapping: {0: 'Unlabeled'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step


,Label,Model Probabilities,Model Class,ADM1,geometry,Dataset name,Farm type,Sentinel,prob_0
0,0,"[0.24285209, 0.7571479]",1,Northern,"POLYGON ((31.69635 16.56403, 31.69648 16.56382...",Sudan,Non-farm,"[[[159.3966434540047, 105.32916491192852, 59.9...",0.242852
1,0,"[0.032209985, 0.96778995]",1,Northern,"POLYGON ((31.7561 16.59074, 31.7561 16.59063, ...",Sudan,Non-farm,"[[[194.47657257659247, 130.85573116454003, 73....",0.032210
2,0,"[0.13263547, 0.8673646]",1,Northern,"POLYGON ((31.61158 17.04083, 31.61164 17.04079...",Sudan,Non-farm,"[[[244.3510896089415, 148.7425196315044, 78.67...",0.132635
3,0,"[0.7517255, 0.24827446]",0,Northern,"POLYGON ((31.41967 18.02514, 31.4197 18.02507,...",Sudan,Non-farm,"[[[195.39575813129483, 135.15770907465375, 82....",0.751725
4,0,"[0.015082599, 0.98491734]",1,Northern,"POLYGON ((31.56502 18.02835, 31.56542 18.02831...",Sudan,Non-farm,"[[[212.23700029774577, 147.9646751113527, 93.8...",0.015083


In [46]:
# Visualize

import geemap.colormaps as cm

def get_fc(df):
  gdf = gpd.GeoDataFrame(df, crs="EPSG:4326")
  fc = geemap.geopandas_to_ee(gdf.drop(columns=["Sentinel"]))
  return fc

boundary_fc = get_fc(results)
buildings_fc = get_fc(results)
pred_0_fc = get_fc(results[results["Model Class"] == 0])
pred_1_fc = get_fc(results[results["Model Class"] == 1])

#get the image extent
def buffer_and_bound(feature, buffer_radius=240):
    return feature.centroid().buffer(buffer_radius, 2).bounds()
pred_0_fc = pred_0_fc.map(buffer_and_bound)
pred_1_fc = pred_1_fc.map(buffer_and_bound)

os.environ["HYBRID"] = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

boundary_viz = {
  'color': 'purple',
  'width': 2,
  'fillColor': '00000000'
}

empty = ee.Image().byte()
probs = empty.paint(**{
  'featureCollection': pred_0_fc,
  'color': 'prob_0',
  'width': 3
    })
fillspalette = cm.get_palette(cmap_name="Spectral_r")

Map = geemap.Map()
Map.centerObject(buildings_fc.first().geometry(), 8)
Map.add_basemap("HYBRID")
Map.addLayer(probs, {'palette': fillspalette, 'min': 0, 'max': 1},\
             'CAFO probability')
Map